Leung Wai Liu <br>
JPMC <br>
July 25, 2022 <br>
Twitter Relabelling WHOLE SENTENCE

In [ ]:
import pandas as pd
import numpy as np
import wordninja
import spacy
from collections import Counter

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
combined_df = pd.read_csv('logs/combined_final.tsv', sep='\t')
combined_df

In [ ]:
# Changing all @user to be B-PER for now
new_label = combined_df['NER Label'].tolist()
new_pos = combined_df['POS Label'].tolist()

for index, row in combined_df.iterrows():
    # retrieving the word of a particular row
    particular_word = row['Word']

    # retrieving the word index in the tweet for verification
    particular_start_index = row['Word Index']
    
    original_sentence = row['Tweet']
    original_sentence_split = original_sentence.split(' ')
    # print(original_sentence_split)

    is_hashtag = False
    encode = ''
    encode_len = 0
    to_feed_sentence = original_sentence
    to_feed_split = original_sentence_split
    for word in range(len(to_feed_split))
    
    
    # if a particular word is a user tag - label it has a person
    if '@user' in particular_word:
        new_label[index] = 'B-PER'
        new_pos[index] = 'User: No change'
    else:
        # if there is a hashtag in a particular word, cut out the hashtag to do further processing
        if '#' in particular_word:
            particular_word = particular_word[1:]
            is_hashtag = True

        # breaking the word into phrases with Viterbo splitting (WordNinja)
        if is_hashtag == True: 
            encode = wordninja.split(particular_word)
            encode_len = len(encode)
            # print('ENCODE LEN FOR HASHTAG:', encode_len)
            to_feed_split = original_sentence_split[:(particular_start_index-1)] + encode + original_sentence_split[particular_start_index:]
            to_feed_sentence = ' '.join(to_feed_split)
            # print('TO FEED SENTENCE', to_feed_sentence)
        else:
            # if not hashtag, then we keep the particular word in the encode and the original sentence split 
            # (breaking one word apart into letters might result in worse results) 
            encode = particular_word
            encode_len = len(encode.split(' '))

        # encoding into spacy document to get the NER entity of each word
        sentence_doc = nlp(to_feed_sentence) # splitted word phrase
        word_entities = [(e.text, e.ent_iob_, e.ent_type_, e.pos_) for e in sentence_doc]

        # check if the length of the entities that fed in matches with the len of entities that came out
        print('-' * 50)
        print('Original Particular word')
        print(particular_word)
        print('To feed sentence')
        print(to_feed_sentence)
        print('To feed split')
        print(to_feed_split)
        print('Word Entities result')
        print(word_entities)
        print("IF LENGTH MATCH AFTER SPACY PASS:", len(word_entities) == len(to_feed_split))
        print('-' * 50)
  
        # retrieving all the labels and pos tags from each phrase (if any)
        label = []
        pos = []

        word_entities_to_verify = word_entities[particular_start_index : (particular_start_index + encode_len)]
        print("PARTICULAR WORD", particular_word)
        print("WORD ENTITIES TO VERIFY", word_entities_to_verify)
        
        # otherwise append all of the NER tags from the split word entity (if there are)
        for numents in range(len(word_entities_to_verify)):
            if word_entities_to_verify[numents][2] != '':
                label.append(word_entities_to_verify[numents][2])
            pos.append(word_entities_to_verify[numents][3])
        
        print(label)
        print(pos)
        exit()
        # Filtering - if the label list is not empty: 
        if label != []:
            # Find the most common label tagging
            label_counter = Counter(label)
            label_most_common = (label_counter.most_common(1))[0][0]
            
            # checking the label itself and then labeling it based on the Tweebank labelling standard
            if label_most_common == 'PERSON':
                new_label[index] = 'B-PER'
            elif label_most_common == 'FAC':
                new_label[index] = 'B-LOC'
            elif label_most_common == 'NORP':
                new_label[index] = 'B-ORG'
            elif label_most_common == 'ORG':
                new_label[index] = 'B-ORG'
            elif label_most_common != 'MONEY' or label_most_common != 'QUANTITY':
                new_label[index] = 'B-MISC'

            pos_counter = Counter(pos)
            pos_most_common = (pos_counter.most_common(1)[0][0])
            new_pos[index] = pos_most_common
            
        else: 
            # If label list is empty, Looking through pos taggings to see if we can classify anything
            # if there is a proper noun, then it can still be classified as miscellaneous
            if 'PROPN' in pos: 
                new_label[index] = 'B-MISC'
                new_pos[index] = 'PROPN'
            elif 'NOUN' in pos:
                new_label[index] = 'B-MISC'
                new_pos[index] = 'NOUN'
            else:
                # if it is not able to find a label, then we find what SPACY thinks the POS tagging is
                pos_counter = Counter(pos)
                pos_most_common = (pos_counter.most_common(1)[0][0])
                new_pos[index] = pos_most_common
 


In [ ]:
new_df = combined_df
new_df['Spacy POS Label'] = new_pos
new_df['Spacy NER Label'] = new_label

new_df = new_df.reindex(columns = ['Word', 'POS Label', 'NER Label', 'Spacy POS Label', 'Spacy NER Label', 'Tweet'])
new_df

In [ ]:
new_df.to_csv('logs/relabelled_data_spacy_pass.tsv', sep='\t', index=False)